<a href="https://colab.research.google.com/github/ArvindReddyC/Sord_analysis/blob/main/newModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cloning from git

In [ ]:
!git clone https://github.com/ArvindReddyC/Sord_analysis.git

Cloning into 'Sord_analysis'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 12), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [ ]:
!pwd

/content


## Modules Needed

In [99]:
import pandas as pd
import numpy as np
import gdown

%load_ext autoreload
%autoreload 2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start of code 

In [ ]:
!touch functions.py

### Important Functions 

In [2]:
def encapsule(df):
    display(df.head(5))
    print( f'shape of df = {df.shape}' )
    return df

def tran(x):
    y = x[1:-1]
    return y.split(',')

def only_activities( df , list_of_activities_to_include ): 
    return df[ df.Activity.isin( list_of_activities_to_include ) ]


def data_wrangle( df ):
    sample = df[['timestamp','raw_sensor_data']];
    sample['parameters'] = sample.raw_sensor_data.apply(tran);
    main = pd.DataFrame( sample.parameters.to_list() , columns = ['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11'] );
    main['timestamp'] = sample['timestamp'];
    main.drop( 'v1' , axis = 1 , inplace = True );
    main.set_index('timestamp' , inplace=True);
    return main;


def removeprocess(x):
    try:
      main , *_  = x.split(' ')
    except (ValueError,AttributeError):
      return x
    return main

def val_counts(df):
    return df['Activity'].value_counts()
'''
Will t
'''
def groupby_avarage(activity_df):
    activity_df['Time'] =  activity_df.Time.astype(int)
    means_df = activity_df.groupby('Time').mean()
    mode_of_activities = []
    for each in activity_df.groupby('Time'):
        mode_of_activities.append(each[1]['Activity'].mode()[0])
    means_df['Activities'] = mode_of_activities
    return means_df


def process_sheet(df, number):
    df = df.fillna(method='ffill')
    df.drop( ['SORD', 'AP time' , 'AP classification'] , axis = 1 , inplace = True )
    df = only_activities( df , ['Standing', 'Sitting', 'Reclining' , 'Walking', 'Sitting (outsrtetched legs)'] )
    df['User_Id'] = number 
    df['Activity'] = df.Activity.apply(removeprocess)
    df.Activity = df.Activity.replace( 'Reclining' , 'Sitting' )  
    display(val_counts(df))
    return df


def process_sheet_edit(df, number):
    df = df.fillna(method='ffill')
    display(val_counts(df))
    df['Activity'] = df.Activity.apply(removeprocess)
    # df.drop( ['SORD', 'AP time' , 'AP classification'] , axis = 1 , inplace = True )
    df = only_activities( df , ['Standing', 'Sitting', 'Reclining' , 'Walking', 'Sitting'] )
    df['User_Id'] = number 
    df.Activity = df.Activity.replace( 'Reclining' , 'Sitting' )  
    display(val_counts(df))
    return df


def drop_un(df):
  try:
    df.drop( ['SORD', 'AP time' , 'AP classification'] , axis = 1 , inplace = True )
  except:
    return df 
  return df

### Reading the csv  

### Sheet one cleaning

In [3]:
sheet_one = pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='01')

In [ ]:
sheet_one = sheet_one.fillna(method='ffill')
sheet_one.drop( ['SORD', 'AP time' , 'AP classification'] , axis = 1 , inplace = True )
sheet_one = only_activities( sheet_one , ['Standing', 'Sitting', 'Reclining' , 'Walking', 'Sitting (outsrtetched legs)'] )
sheet_one['User_Id'] = 1 
sheet_one['Activity'] = sheet_one.Activity.apply(removeprocess)
sheet_one.Activity = sheet_one.Activity.replace( 'Reclining' , 'Sitting' )


### Sheet two Cleaning

In [5]:
sheet_two = pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='02')

In [6]:
sheet_two = encapsule(sheet_two.fillna(method='ffill'))
sheet_two.drop( ['SORD', 'AP time' , 'AP classification'] , axis = 1 , inplace = True )
sheet_two = only_activities( sheet_two , ['Standing', 'Sitting', 'Reclining' , 'Walking', 'Sitting (outsrtetched legs)'] )
sheet_two['User_Id'] = 2
sheet_two['Activity'] = sheet_two.Activity.apply(removeprocess)
sheet_two.Activity = sheet_two.Activity.replace( 'Reclining' , 'Sitting' )



,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.098994,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal error,NaN,NaN,NaN
1,0.197000,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal error,NaN,NaN,NaN
2,0.397000,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal error,NaN,NaN,NaN
3,0.796000,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal error,NaN,NaN,NaN
4,0.995000,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal error,NaN,NaN,NaN


shape of df = (139166, 15)


In [21]:
val_counts(sheet_two)

Sitting     63069
Standing    29029
Walking      8962
Name: Activity, dtype: int64

### Sheet three 

In [7]:
sheet_three = pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='03')


In [8]:
sheet_three = process_sheet(sheet_three,3)

Sitting     66504
Standing    28078
Walking      8979
Name: Activity, dtype: int64

### Sheet Five

In [9]:
sheet_five = pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='05')


In [10]:
sheet_five_copy = sheet_five.copy( deep = True)

In [11]:
sheet_five =  process_sheet(sheet_five,5)


Standing    30396
Sitting     11330
Walking      7008
Name: Activity, dtype: int64

### Sheet six

In [12]:
sheet_six = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='06'))


,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.062995,172.23,-0.85,-0.91,9.30,0.00,0.00,0.06,-3.56,-21.38,76.56,Random,NaN,NaN,NaN
1,0.158000,172.23,-0.88,-0.91,9.31,-0.12,0.00,-0.06,-3.25,-21.00,76.56,NaN,NaN,NaN,NaN
2,0.169000,172.23,-0.87,-0.93,9.32,0.00,0.19,-0.06,-3.56,-21.38,76.56,NaN,NaN,NaN,NaN
3,0.187000,172.23,-0.88,-0.90,9.34,0.06,-0.12,0.06,-3.56,-21.00,76.06,NaN,NaN,NaN,NaN
4,0.187000,172.23,-0.85,-0.92,9.30,0.12,0.06,-0.06,-3.56,-21.00,76.19,NaN,NaN,NaN,NaN


shape of df = (137687, 15)


In [13]:
sheet_six = process_sheet(sheet_six,6)

Sitting     50789
Standing    29096
Walking     10396
Name: Activity, dtype: int64

### Sheet 07

In [14]:
sheet_seven = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='07'))
sheet_seven = process_sheet_edit(sheet_seven,7)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,145.06,166.27,-2.19,-0.75,9.50,-0.06,-0.06,0.06,-19.00,-5.75,54.25,Sitting,NaN,NaN,NaN
1,145.08,166.27,-2.21,-0.76,9.46,-0.06,0.00,0.19,-19.00,-5.38,52.75,NaN,NaN,NaN,NaN
2,145.11,166.27,-2.21,-0.79,9.47,0.12,0.00,0.31,-19.00,-5.38,54.00,NaN,NaN,NaN,NaN
3,145.13,166.27,-2.23,-0.78,9.51,0.25,0.00,0.19,-18.19,-5.75,54.00,NaN,NaN,NaN,NaN
4,145.19,166.27,-2.24,-0.78,9.50,0.06,0.00,0.00,-18.56,-6.06,54.00,NaN,NaN,NaN,NaN


shape of df = (111975, 15)


Standing                                                                                                                               6608
Prepared for simple resistance exercises (discard)                                                                                     6155
Sitting 2                                                                                                                              4669
Sitting 4                                                                                                                              4427
Standing 4                                                                                                                             4183
Discard                                                                                                                                4122
Prepared to walk on treadmil                                                                                                           4046
Standing 3          

Sitting     46192
Standing    21096
Walking      7530
Name: Activity, dtype: int64

### Sheet nine

In [15]:
sheet_nine = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='09'))
sheet_nine = process_sheet(sheet_nine,9)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.025005,163.57,-2.59,-0.15,9.72,37.50,68.75,25.00,-2.75,-14.00,32.69,Random,NaN,NaN,NaN
1,0.056005,161.43,-3.87,0.99,9.22,-1.88,-15.50,11.69,-4.00,-12.56,31.88,NaN,NaN,NaN,NaN
2,0.073003,161.81,-3.00,0.75,9.94,-14.00,30.50,72.00,-4.69,-12.19,31.88,NaN,NaN,NaN,NaN
3,0.098008,160.71,-3.60,0.47,10.64,-68.38,64.56,86.44,-4.69,-12.19,31.88,NaN,NaN,NaN,NaN
4,0.117010,158.06,-3.59,-0.11,8.81,-51.12,90.75,31.19,-5.38,-11.88,32.00,NaN,NaN,NaN,NaN


shape of df = (116612, 15)


Sitting     41708
Standing    24381
Walking      8507
Name: Activity, dtype: int64

### Sheet ten

In [16]:
sheet_ten = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='10'))
sheet_ten = process_sheet(sheet_ten,10)


,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.080999,160.19,-0.70,4.09,8.60,-42.31,-34.06,-16.00,-24.25,8.25,38.00,Sitting,NaN,NaN,NaN
1,0.148010,161.62,-2.15,1.13,9.08,45.50,-14.75,8.50,-22.50,9.75,38.69,NaN,NaN,NaN,NaN
2,0.173000,161.07,-1.16,2.76,8.40,-16.69,-10.31,-3.25,-22.06,10.50,38.38,NaN,NaN,NaN,NaN
3,0.186010,161.99,-1.75,2.72,10.18,-40.69,2.38,-17.81,-22.06,10.06,38.75,NaN,NaN,NaN,NaN
4,0.204000,162.38,-1.99,2.17,8.61,27.38,0.75,11.19,-21.06,10.06,38.75,NaN,NaN,NaN,NaN


shape of df = (120642, 15)


Sitting     52269
Standing    23217
Walking      8721
Name: Activity, dtype: int64

### Sheet eleven

In [17]:
sheet_eleven = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='11'))
sheet_eleven = process_sheet(sheet_eleven,11)



,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.026001,173.15,-0.98,-0.69,9.33,0.00,-0.06,-0.06,-15.06,12.88,28.25,Random,NaN,NaN,NaN
1,0.095996,173.15,-0.99,-0.71,9.32,-0.06,-0.06,-0.06,-15.06,13.56,28.25,NaN,NaN,NaN,NaN
2,0.157000,173.15,-0.98,-0.70,9.30,0.00,-0.19,0.00,-14.25,13.56,28.25,NaN,NaN,NaN,NaN
3,0.186000,173.15,-0.97,-0.70,9.31,0.06,-0.12,0.06,-15.06,12.88,29.50,NaN,NaN,NaN,NaN
4,0.279000,173.15,-0.97,-0.70,9.34,-0.06,0.00,-0.12,-15.38,13.19,28.25,NaN,NaN,NaN,NaN


shape of df = (131368, 15)


Sitting     51000
Standing    27242
Walking      9057
Name: Activity, dtype: int64

### Sheet 12

In [18]:
sheet_twelve = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='12'))
sheet_twelve = process_sheet_edit(sheet_twelve,12)


,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.16300,90.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Preparation (discard),NaN,NaN,NaN
1,0.21701,90.00,6.99,0.00,6.99,0.00,43.69,0.00,16.19,-16.00,126.88,NaN,NaN,NaN,NaN
2,0.22601,172.23,-1.18,-0.44,10.65,18.12,-13.75,18.94,-16.38,8.19,45.56,NaN,NaN,NaN,NaN
3,0.23200,172.67,-0.69,-0.60,10.43,18.12,9.69,9.94,-16.38,10.00,45.19,NaN,NaN,NaN,NaN
4,0.26400,172.67,-1.24,-0.72,10.13,25.88,9.94,12.88,-15.69,9.88,44.69,NaN,NaN,NaN,NaN


shape of df = (116067, 15)


Sitting                                                          29404
Standing                                                         25183
Reclining                                                        11379
Lying (face up)                                                   4204
Walking (fast)                                                    4095
Walking (normal)                                                  4095
Sitting (outstretched legs)                                       3887
Prepared for simple resistance exercises (discard)                3725
Discard                                                           3671
Lying (face down)                                                 3444
Lying (left shoulder)                                             3428
Lying (right shoulder)                                            3340
Sitting 2                                                         2698
Sitting 1                                                         2261
single

Sitting     49814
Standing    25183
Walking      9971
Name: Activity, dtype: int64

### sheet 13

In [19]:
sheet_thirteen = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='13'))
sheet_thirteen = process_sheet_edit(sheet_thirteen,13)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.040002,160.89,-2.61,3.15,8.91,-27.06,11.88,49.06,-23.38,-4.69,42.38,Random,NaN,NaN,NaN
1,0.058006,160.89,-2.66,3.28,8.75,-11.06,9.69,34.62,-23.38,-4.69,42.38,NaN,NaN,NaN,NaN
2,0.114000,160.54,-2.71,3.30,8.59,-11.19,4.94,15.38,-23.56,-5.56,43.25,NaN,NaN,NaN,NaN
3,0.168000,159.85,-2.85,3.57,8.69,-5.44,-0.31,3.00,-24.19,-5.56,42.19,NaN,NaN,NaN,NaN
4,0.177010,159.18,-2.81,3.36,8.64,2.00,-2.19,-16.50,-24.69,-4.75,42.19,NaN,NaN,NaN,NaN


shape of df = (123695, 15)


Standing                              26526
Sitting                               19113
Sitting 1                             13944
Discard                               12778
Walking                               10204
Reclining                              9798
Lying (face down)                      3780
Sitting                                3738
Sitting (outstretched legs)            3731
Lying (face up)                        3545
Lying (left shoulder)                  3528
Lying (right shoulder)                 3434
single leg straight leg kick backs     2265
calf raises                            1725
half squat                             1294
Random                                 1170
Walking (random)                        940
Walk (random)                           785
Switch                                  682
half squats                             631
Turned to left shoudler                  84
Name: Activity, dtype: int64

Sitting     50324
Standing    26526
Walking     11144
Name: Activity, dtype: int64

### Sheet 15

In [20]:
sheet_fifteen = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='15'))
sheet_fifteen = process_sheet_edit(sheet_fifteen,15)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,0.22201,90.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
1,0.38500,90.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
2,0.42500,90.00,0.00,9.77,-174.64,126.88,2.88,5.62,-13.75,10.38,48.56,NaN,NaN,NaN,NaN
3,0.45001,176.34,-0.57,0.79,9.53,-14.06,1.25,6.50,-14.19,9.75,48.56,NaN,NaN,NaN,NaN
4,0.52800,176.34,-0.47,0.74,9.78,-6.62,6.31,5.81,-14.19,9.69,48.19,NaN,NaN,NaN,NaN


shape of df = (114876, 15)


Sitting                                               30781
Standing                                              23512
Reclining                                             10289
Walking (on treadmil)                                  8825
Discard                                                5948
Prepared for simple resistance exercises (discard)     3996
Sitting (outstretched legs)                            3883
Lying (face up)                                        3600
Sitting                                                3511
Lying (right shoulder)                                 3476
Lying (face down)                                      3356
Lying (left shoulder)                                  3333
single leg straight leg kick backs                     2430
half squat                                             1410
calf raises                                            1367
calf rises                                              750
half squats                             

Sitting     48544
Standing    23512
Walking      9432
Name: Activity, dtype: int64

### Sheet Sixteen

In [21]:
sheet_sixteen = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='16'))
sheet_sixteen = process_sheet_edit(sheet_sixteen,16)


,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity
0,0.023999,160.71,-2.50,-1.73,9.23,1.94,-0.44,-2.38,-24.56,-0.69,43.38,Preparation (discard)
1,0.083997,160.71,-2.64,-1.77,9.29,1.38,-0.62,-3.06,-24.56,-0.69,41.06,NaN
2,0.137000,160.89,-2.76,-1.83,9.55,3.25,-0.44,-2.00,-23.50,-0.69,41.06,NaN
3,0.150000,160.89,-2.66,-1.97,9.37,4.62,-0.31,-2.31,-23.50,-1.06,43.00,NaN
4,0.248000,161.07,-2.62,-2.08,9.07,3.44,0.31,-2.31,-23.50,-0.69,42.25,NaN


shape of df = (120228, 12)


Sitting                                                          32831
Standing                                                         17117
Reclining                                                        14472
Standing                                                          6753
Discard                                                           6636
Prepared for simple resistance exercises (discard)                4410
Sitting (outstretched legs)                                       3900
Walking (normal)                                                  3900
Lying (left shoulder)                                             3747
Lying (face down)                                                 3644
Lying (face up)                                                   3457
Walking (fast)                                                    3424
Lying (right shoulder)                                            3353
single leg straight leg kick backs                                2276
half s

Sitting     51203
Standing    23870
Walking      8914
Name: Activity, dtype: int64

### Sheet Seventeen

In [22]:
sheet_seventeen = encapsule(pd.read_excel('/content/drive/MyDrive/SORD combined_Feb_2022.xlsx' , sheet_name='17'))
sheet_seventeen = process_sheet_edit(sheet_seventeen,17)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity,SORD,AP time,AP classification
0,150.00,162.77,-2.57,-0.41,8.94,15.88,2.56,-4.88,-28.75,-2.19,57.75,Sitting 1,NaN,NaN,NaN
1,150.03,162.57,-2.98,0.45,9.37,13.06,3.69,-1.75,-28.75,-2.56,56.50,NaN,NaN,NaN,NaN
2,150.05,162.57,-3.09,0.84,9.65,-2.38,2.62,0.00,-28.75,-1.50,57.00,NaN,NaN,NaN,NaN
3,150.11,162.38,-2.65,-0.41,9.06,-5.19,4.81,-3.25,-28.75,-1.50,57.00,NaN,NaN,NaN,NaN
4,150.12,162.18,-2.76,-0.49,9.36,4.00,5.25,1.25,-28.75,-2.19,56.19,NaN,NaN,NaN,NaN


shape of df = (110897, 15)


Standing                                                         10003
Prepared for simple resistance exercises (discard)                4591
Prepared to walk on treadmil                                      4446
Sitting (outstretched legs)                                       3795
Sitting 8                                                         3696
Standing 5                                                        3617
Standing 4                                                        3608
Sitting 6                                                         3584
Sitting 9                                                         3571
Sitting 3                                                         3506
Sitting 5                                                         3480
Sitting 7                                                         3474
Lying (face up)                                                   3466
Lying (right shoulder)                                            3465
Walkin

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Sitting     45493
Standing    23819
Walking      7465
Name: Activity, dtype: int64

In [ ]:
raw_df.head(5)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity
0,145.06,166.27,-2.19,-0.75,9.50,-0.06,-0.06,0.06,-19.00,-5.75,54.25,Sitting 1
1,145.08,166.27,-2.21,-0.76,9.46,-0.06,0.00,0.19,-19.00,-5.38,52.75,NaN
2,145.11,166.27,-2.21,-0.79,9.47,0.12,0.00,0.31,-19.00,-5.38,54.00,NaN
3,145.13,166.27,-2.23,-0.78,9.51,0.25,0.00,0.19,-18.19,-5.75,54.00,NaN
4,145.19,166.27,-2.24,-0.78,9.50,0.06,0.00,0.00,-18.56,-6.06,54.00,NaN


### Preprocessing

In [ ]:
print(raw_df.shape)
raw_df.Activity.replace(np.NaN,1,inplace = True)
indexes = raw_df[ raw_df.Activity != 1   ].index
indexes
# filling the columns
for each in range((len(indexes) - 1) ):
  raw_df.loc[indexes[each]:(indexes[ each + 1]-1)]['Activity'] =  raw_df.loc[indexes[each]]['Activity']
raw_df.loc[91909:95403,'Activity'] = 'Walking 3'

'''
The below function can replace the above setsp 
'''
df.fillna(method='ffill')

(95403, 12)


In [ ]:
raw_df.Activity.value_counts().index

Index(['Standing', 'Sitting 2', 'Sitting 4', 'Standing 4',
       'Prepared to walk on treadmil', 'Standing 3', 'Lying 1', 'Sitting 5',
       'Sitting (oustretched legs)', 'Walking 3', 'Sitting 6', 'Sitting 9',
       'Lying 2', 'Sitting 8', 'Walking 2', 'Sitting 1', 'Lying 4', 'Lying 3',
       'Standing 2', 'Reclining 2', 'Reclining 3', 'Sitting 3 ', 'Reclining 1',
       'Sitting 7', 'Standing 5', 'Walking 1',
       'Switch from normal to fast walking',
       'Moved towards matt and prepared to sit with outstretched legs',
       'Switched to face down', 'Switch to lie down',
       'Turned to left shoudler', 'Switch',
       'Moved towards the wall to stand (discard)',
       'Switched to face down (Turned)',
       'Moved towards the same spot for standing', 'Switch to standing',
       'Switching legs', 'From standing to sitting',
       'Switch to right leg over knee', 'Switch to sitting',
       'Switch to lean on left shoulder', 'Switch from standing to walking',
       'Ut

In [ ]:
copy_raw_df.head(5)

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activity
0,145.06,166.27,-2.19,-0.75,9.50,-0.06,-0.06,0.06,-19.00,-5.75,54.25,Sitting 1
1,145.08,166.27,-2.21,-0.76,9.46,-0.06,0.00,0.19,-19.00,-5.38,52.75,Sitting 1
2,145.11,166.27,-2.21,-0.79,9.47,0.12,0.00,0.31,-19.00,-5.38,54.00,Sitting 1
3,145.13,166.27,-2.23,-0.78,9.51,0.25,0.00,0.19,-18.19,-5.75,54.00,Sitting 1
4,145.19,166.27,-2.24,-0.78,9.50,0.06,0.00,0.00,-18.56,-6.06,54.00,Sitting 1


In [ ]:
copy_raw_df = raw_df.copy( deep = 'True')

In [ ]:
activity_df = raw_df[ ~raw_df.Activity.isin( [ 'Switch from normal to fast walking',
       'Moved towards matt and prepared to sit with outstretched legs',
       'Switched to face down', 'Switch to lie down',
       'Turned to left shoudler', 'Switch',
       'Moved towards the wall to stand (discard)',
       'Switched to face down (Turned)',
       'Moved towards the same spot for standing', 'Switch to standing',
       'Switching legs', 'From standing to sitting',
       'Switch to right leg over knee', 'Switch to sitting',
       'Switch to lean on left shoulder', 'Switch from standing to walking',
       'Uturn to stand',
       'Switch from moving left foot to right foot (This subject started fidgeting his left leg first- right leg is first in the protocol:)','Prepared to walk on treadmil','Lying 1','Lying 4','Lying 3','Lying 2'] ) ]

In [ ]:
def removeprocess(x):
    try:
      main , *_  = x.split(' ')
    except ValueError:
      return x
    return main
activity_df.Activity = activity_df.Activity.apply(removeprocess)
activity_df.Activity = activity_df.Activity.replace( 'Reclining' , 'Sitting' )

In [ ]:
activity_df.Activity.value_counts()

Sitting     46192
Standing    21096
Walking      7530
Name: Activity, dtype: int64

In [ ]:
#Check Point
copy_activity_df_df = activity_df.copy( deep = 'True')


In [ ]:
activity_df['Time'] =  activity_df.Time.astype(int)
means_df = activity_df.groupby('Time').mean()
mode_of_activities = []
for each in activity_df.groupby('Time'):
    mode_of_activities.append(each[1]['Activity'].mode()[0])
means_df['Activities'] = mode_of_activities
means_df['Activities'].value_counts()
    

In [ ]:
means_df

,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Activities
Time,,,,,,,,,,,
145,166.270000,-2.207407,-0.762593,9.497778,-0.094444,-0.057037,-0.023333,-18.681111,-5.792593,54.161111,Sitting
146,166.270000,-2.210000,-0.760741,9.498148,-0.073333,-0.050741,-0.027037,-18.639259,-6.061481,54.127407,Sitting
147,166.270000,-2.205185,-0.763333,9.496667,0.039259,-0.017778,-0.015556,-18.740370,-5.791481,53.940000,Sitting
148,166.270000,-2.198571,-0.772143,9.497143,0.031071,-0.002500,0.000000,-18.745000,-6.120714,53.895357,Sitting
149,166.270000,-2.196667,-0.775556,9.499630,0.005185,-0.035926,-0.028148,-18.449630,-6.055185,54.298148,Sitting
...,...,...,...,...,...,...,...,...,...,...,...
3703,53.410385,-10.901923,0.548462,0.672692,-5.977692,4.585769,10.339615,-53.371923,-26.087308,-9.523846,Walking
3704,52.474815,-10.627778,0.649259,-0.981481,12.484815,-0.832963,12.614444,-53.528889,-26.617037,-8.483704,Walking
3705,51.481154,-10.300769,0.778462,0.206538,-13.488462,1.052692,11.773077,-53.732308,-26.334231,-9.202308,Walking


In [ ]:
means_df['Activities'].value_counts()

Sitting     1714
Standing     801
Walking      288
Name: Activities, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score

In [106]:
def preprocessing():
  # for each in [sheet_one, sheet_two, sheet_three, sheet_five, sheet_six, sheet_seven, sheet_nine, sheet_ten, sheet_eleven, sheet_twelve, sheet_thirteen, sheet_fifteen, sheet_sixteen, sheet_seventeen]:
  # print(each.shape)
  feture_selection_raw = []
  for each in [sheet_one, sheet_two, sheet_three, sheet_five, sheet_six, sheet_seven, sheet_nine, sheet_ten, sheet_eleven, sheet_twelve, sheet_thirteen, sheet_fifteen, sheet_sixteen, sheet_seventeen]:
    feture_selection_raw.append(drop_un(each))
  for each in feture_selection_raw:
    print(each.shape)
  return feture_selection_raw

In [78]:
def make_windows(df):
  lists = []
  print(int(df.sort_values('Time').Time.max()))
  for i in range( int(df.sort_values('Time').Time.min()) ,int(df.sort_values('Time').Time.max()) - 6 , 5  ):
      window =  df[ (df.Time >= i) & (df.Time <= (i + 5) )  ]
      window_mean = window.mean()
      try:
        window_mean['Activity'] = window.Activity.mode()[0]
      except:
        continue
      lists.append(window_mean)
  return pd.DataFrame(lists)


In [ ]:
def creating_df(feture_selection_raw):
  lists_  = []  
  for each in feture_selection_raw:
    lists_.append(make_windows(each))
  main_df = encapsule(pd.concat( lists_ ))
  main_df = main_df.sort_values(by = ['User_Id', 'Time'], ignore_index=True)
  return  main_df
main_df = creating_df(feture_selection_raw)


In [86]:
main_df.Activity.value_counts()

Sitting     5096
Standing    2945
Walking      955
Name: Activity, dtype: int64

In [ ]:
# list_of_dfs = []
# for each in [sheet_one, sheet_two, sheet_three, sheet_five, sheet_six, sheet_seven, sheet_nine, sheet_ten, sheet_eleven, sheet_twelve, sheet_thirteen, sheet_fifteen, sheet_sixteen, sheet_seventeen]:
#     list_of_dfs.append( groupby_avarage(each) )


# concat = pd.concat(list_of_dfs, axis = 0 )
# concat['Time_stamp'] = concat.index
# df_main = concat.sort_values(by = ['User_Id', 'Time_stamp'], ignore_index=True)
  

In [90]:
split = int(((main_df.shape[0]) * 0.75))
train =  main_df.iloc[ 0:split , :  ] 
test = main_df.iloc[ split: , :  ] 

In [92]:
train.Activity.value_counts()

Sitting     3882
Standing    2188
Walking      677
Name: Activity, dtype: int64

In [139]:
main_df.to_csv('dataset_5_sec.csv')

### Modeling 

In [179]:
main_df

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,User_Id,Activity
0,2.869542,162.382288,-2.570932,-1.436780,9.255254,-0.191102,0.456780,-0.073898,-25.810847,-2.242881,46.616780,1.0,Sitting
1,7.493096,163.604222,-2.511481,-1.087926,9.353407,-0.070296,-0.076667,0.020222,-26.028444,0.315630,47.569778,1.0,Sitting
2,12.485081,163.736765,-2.503971,-1.064632,9.351544,0.241103,0.035294,0.072941,-25.876691,0.566985,47.474779,1.0,Sitting
3,17.459985,161.418235,-2.929191,-1.009044,9.247059,0.093824,0.923824,0.333676,-28.525662,0.618971,46.099338,1.0,Sitting
4,22.468366,160.753060,-3.084925,-0.839851,9.215224,0.274552,-0.211493,0.163955,-29.361045,1.617388,45.619627,1.0,Sitting
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8991,3787.531655,96.668561,-10.070072,0.943669,-1.955683,0.069928,8.441223,12.706043,-54.278849,-25.005683,-14.085036,17.0,Walking
8992,3792.489855,94.374493,-9.855725,0.600145,-1.952101,5.690145,-2.883768,15.216594,-53.456232,-26.023043,-15.839348,17.0,Walking
8993,3797.513869,96.629489,-10.100146,0.515547,-2.223285,-4.122555,-6.905109,13.462117,-54.886934,-25.983504,-13.183723,17.0,Walking
8994,3802.493478,95.108551,-9.475290,0.703188,-2.130362,4.868116,8.705290,14.307174,-54.797826,-25.364493,-14.374493,17.0,Walking


In [97]:
clf = RandomForestClassifier( random_state = 0 )
clf.fit( train[ train.columns.drop(['Activity','User_Id','Time'])  ] , train.Activity )




RandomForestClassifier(random_state=0)

In [101]:
preds = clf.predict(test[ test.columns.drop(['Activity','User_Id','Time'])  ] )
accuracy_score( test.Activity , preds ) * 100

99.64428634948867

In [104]:
from collections import Counter
Counter(preds)

Counter({'Sitting': 1214, 'Standing': 753, 'Walking': 282})

In [ ]:
print( f'max value in Angle = {np.max( df_main.Angle)}' )
print( f'min value in Angle = {np.min( df_main.Angle)}' )




max value in Angle = 180.0
min value in Angle = 33.183928571428574


### Testing on the new set

In [180]:
sample = encapsule(pd.read_csv('new_test_data.csv'))
main = encapsule(data_wrangle(sample))

,id,user_id,email_id,raw_sensor_data,predicted_activity,actual_activity,timestamp,createdAt,updatedAt
0,458813,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[1.626615,8.192087525665539,-0.99,0.8,-9.53,-0...",IN_PROGRESS,NaN,2022-02-13T14:32:00.378Z,2022-02-14T01:32:45.325Z,2022-02-14T01:32:45.325Z
1,458814,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[2.114199,8.192087525665539,-0.99,0.81,-9.6,-0...",IN_PROGRESS,NaN,2022-02-13T14:32:00.866Z,2022-02-14T01:32:45.332Z,2022-02-14T01:32:45.332Z
2,458815,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[2.598735,8.192087525665539,-1.01,0.84,-9.58,-...",IN_PROGRESS,NaN,2022-02-13T14:32:01.350Z,2022-02-14T01:32:45.340Z,2022-02-14T01:32:45.340Z
3,458816,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[3.08846,8.192087525665539,-0.99,0.78,-9.57,0....",IN_PROGRESS,NaN,2022-02-13T14:32:01.840Z,2022-02-14T01:32:45.348Z,2022-02-14T01:32:45.348Z
4,458817,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[3.573518,8.192087525665539,-0.99,0.82,-9.55,0...",IN_PROGRESS,NaN,2022-02-13T14:32:02.324Z,2022-02-14T01:32:45.363Z,2022-02-14T01:32:45.363Z


shape of df = (8240, 9)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11
timestamp,,,,,,,,,,
2022-02-13T14:32:00.378Z,8.192087525665539,-0.99,0.8,-9.53,-0.0625,0.0625,0,16.1875,-19.375,-61.875
2022-02-13T14:32:00.866Z,8.192087525665539,-0.99,0.81,-9.6,-0.0625,0,-0.0625,15.75,-18.6875,-62.75
2022-02-13T14:32:01.350Z,8.192087525665539,-1.01,0.84,-9.58,-0.1875,-0.125,-0.25,16.5,-19.375,-62.25
2022-02-13T14:32:01.840Z,8.192087525665539,-0.99,0.78,-9.57,0.3125,-0.0625,-0.0625,16.1875,-19.0625,-62.75
2022-02-13T14:32:02.324Z,8.192087525665539,-0.99,0.82,-9.55,0.0625,0.0625,0,16.5,-19.0625,-61.875


shape of df = (8240, 10)


In [188]:
daate = pd.to_datetime( main.index  )
main['Second'] =  daate.second
main['Minutes'] =  daate.minute
main['Hour'] = daate.hour

In [189]:
cols = list(train.columns[1:-2])
cols.append('Second')
cols.append('Minutes')
cols.append('Hour')

In [190]:
main.columns = cols

In [191]:
main

,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Second,Minutes,Hour
timestamp,,,,,,,,,,,,,
2022-02-13T14:32:00.378Z,8.192087525665539,-0.99,0.8,-9.53,-0.0625,0.0625,0,16.1875,-19.375,-61.875,0,32,14
2022-02-13T14:32:00.866Z,8.192087525665539,-0.99,0.81,-9.6,-0.0625,0,-0.0625,15.75,-18.6875,-62.75,0,32,14
2022-02-13T14:32:01.350Z,8.192087525665539,-1.01,0.84,-9.58,-0.1875,-0.125,-0.25,16.5,-19.375,-62.25,1,32,14
2022-02-13T14:32:01.840Z,8.192087525665539,-0.99,0.78,-9.57,0.3125,-0.0625,-0.0625,16.1875,-19.0625,-62.75,1,32,14
2022-02-13T14:32:02.324Z,8.192087525665539,-0.99,0.82,-9.55,0.0625,0.0625,0,16.5,-19.0625,-61.875,2,32,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-13T15:41:36.500Z,9.342810245644976,-1.16,0.91,-9.65,0.1875,-0.0625,-0.25,21.5,-14,-69.875,36,41,15
2022-02-13T15:41:36.983Z,9.342810245644976,-1.19,0.92,-9.66,0,0,0,21.875,-13.5625,-69.5,36,41,15
2022-02-13T15:41:37.475Z,9.342810245644976,-1.18,0.92,-9.66,0,0,0.125,21.875,-13.1875,-69.5,37,41,15


In [192]:
main.reset_index(drop=True, inplace = True)

In [193]:
main

,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Second,Minutes,Hour
0,8.192087525665539,-0.99,0.8,-9.53,-0.0625,0.0625,0,16.1875,-19.375,-61.875,0,32,14
1,8.192087525665539,-0.99,0.81,-9.6,-0.0625,0,-0.0625,15.75,-18.6875,-62.75,0,32,14
2,8.192087525665539,-1.01,0.84,-9.58,-0.1875,-0.125,-0.25,16.5,-19.375,-62.25,1,32,14
3,8.192087525665539,-0.99,0.78,-9.57,0.3125,-0.0625,-0.0625,16.1875,-19.0625,-62.75,1,32,14
4,8.192087525665539,-0.99,0.82,-9.55,0.0625,0.0625,0,16.5,-19.0625,-61.875,2,32,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8235,9.342810245644976,-1.16,0.91,-9.65,0.1875,-0.0625,-0.25,21.5,-14,-69.875,36,41,15
8236,9.342810245644976,-1.19,0.92,-9.66,0,0,0,21.875,-13.5625,-69.5,36,41,15
8237,9.342810245644976,-1.18,0.92,-9.66,0,0,0.125,21.875,-13.1875,-69.5,37,41,15
8238,9.342810245644976,-1.18,0.92,-9.67,0.0625,-0.0625,0.0625,22.25,-14,-68.6875,38,41,15


In [194]:
main = main.astype(float)

In [121]:
print( f'max value in Angle = {np.max(main.Angle)}' )
print( f'min value in Angle = {np.min( main.Angle)}' )

AttributeError: ignored

In [128]:
meaned_main =  main.groupby(['Hour','Time'])


In [138]:
len(meaned_main)

1186

In [145]:
lists_2  = []  
for each in meaned_main:
  lists_2.append( make_windows(each[1]))
  

In [178]:
train

,Time,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,User_Id,Activity
0,2.869542,162.382288,-2.570932,-1.436780,9.255254,-0.191102,0.456780,-0.073898,-25.810847,-2.242881,46.616780,1.0,Sitting
1,7.493096,163.604222,-2.511481,-1.087926,9.353407,-0.070296,-0.076667,0.020222,-26.028444,0.315630,47.569778,1.0,Sitting
2,12.485081,163.736765,-2.503971,-1.064632,9.351544,0.241103,0.035294,0.072941,-25.876691,0.566985,47.474779,1.0,Sitting
3,17.459985,161.418235,-2.929191,-1.009044,9.247059,0.093824,0.923824,0.333676,-28.525662,0.618971,46.099338,1.0,Sitting
4,22.468366,160.753060,-3.084925,-0.839851,9.215224,0.274552,-0.211493,0.163955,-29.361045,1.617388,45.619627,1.0,Sitting
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6742,1291.513043,150.740000,-4.481812,-1.672971,8.510435,0.047174,0.047899,-0.036667,-40.803261,-8.275942,43.983478,13.0,Sitting
6743,1296.500000,150.819708,-4.481314,-1.676058,8.508978,0.054307,0.012044,-0.005474,-40.777810,-8.280730,43.935693,13.0,Sitting
6744,1301.486466,150.860000,-4.477820,-1.683910,8.508346,-0.026617,-0.014286,0.001805,-40.800752,-8.325489,43.862707,13.0,Sitting
6745,1306.501471,150.860000,-4.479044,-1.685809,8.506691,-0.011103,0.024926,-0.006176,-40.899853,-8.206618,43.999265,13.0,Sitting


In [195]:
def prep(df):
  df =  df.sort_values(['Hour','Minutes', 'Second'])
  lists = []
  for i in range(0,len(df) - 13,11):
      window_mean = df.loc[i:i+11].mean()
      lists.append(window_mean)
  return pd.DataFrame(lists)


In [196]:
meaned_main = encapsule(prep(main))

,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Second,Minutes,Hour
0,8.192088,-0.988333,0.801667,-9.568333,-0.031250,0.000000,-0.093750,16.114583,-18.776042,-62.453125,2.500000,32.0,14.0
1,8.192088,-1.008333,0.835833,-9.569167,-0.130208,-0.093750,-0.072917,16.135417,-18.453125,-62.651042,8.666667,32.0,14.0
2,8.192088,-1.019167,0.846667,-9.560000,0.026042,0.130208,0.072917,16.270833,-18.109375,-62.578125,14.166667,32.0,14.0
3,7.876297,-1.022500,0.803333,-9.550833,0.343750,-0.078125,0.203125,16.104167,-18.760417,-62.755208,19.500000,32.0,14.0
4,7.771033,-0.984167,0.787500,-9.556667,0.078125,0.036458,0.130208,16.411458,-18.895833,-62.708333,25.000000,32.0,14.0


shape of df = (748, 13)


In [197]:
preds = clf.predict( meaned_main[ meaned_main.columns.drop(['Second','Minutes','Hour']) ] )


In [ ]:
meaned_main.shape

(60, 10)

In [198]:
from collections import Counter
Counter(preds)

Counter({'Standing': 686, 'Walking': 62})

In [200]:
meaned_main['Activity'] = preds

In [201]:
meaned_main

,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Second,Minutes,Hour,Activity
0,8.192088,-0.988333,0.801667,-9.568333,-0.031250,0.000000,-0.093750,16.114583,-18.776042,-62.453125,2.500000,32.0,14.0,Standing
1,8.192088,-1.008333,0.835833,-9.569167,-0.130208,-0.093750,-0.072917,16.135417,-18.453125,-62.651042,8.666667,32.0,14.0,Standing
2,8.192088,-1.019167,0.846667,-9.560000,0.026042,0.130208,0.072917,16.270833,-18.109375,-62.578125,14.166667,32.0,14.0,Standing
3,7.876297,-1.022500,0.803333,-9.550833,0.343750,-0.078125,0.203125,16.104167,-18.760417,-62.755208,19.500000,32.0,14.0,Standing
4,7.771033,-0.984167,0.787500,-9.556667,0.078125,0.036458,0.130208,16.411458,-18.895833,-62.708333,25.000000,32.0,14.0,Standing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,9.342810,-1.205000,0.921667,-9.662500,0.036458,0.005208,0.062500,22.270833,-14.026042,-69.364583,5.416667,41.0,15.0,Standing
744,9.342810,-1.200833,0.920833,-9.665833,-0.015625,0.052083,-0.031250,22.145833,-13.744792,-69.291667,11.666667,41.0,15.0,Standing
745,9.342810,-1.196667,0.927500,-9.660833,0.010417,-0.026042,-0.005208,22.145833,-13.875000,-69.234375,17.500000,41.0,15.0,Standing
746,9.342810,-1.203333,0.928333,-9.654167,-0.036458,-0.041667,-0.031250,22.203125,-14.093750,-69.187500,23.083333,41.0,15.0,Standing


## Trying a model for standing and Walking.

In [208]:
#display(meaned_main[ (meaned_main.Minutes >= 32 ) & (meaned_main.Minutes <= 47 ) & (meaned_main.Hour == 14 ) ])
display(meaned_main[ (meaned_main.Minutes >= 47 ) & (meaned_main.Minutes <= 58 ) & (meaned_main.Hour == 14 ) ].Activity.value_counts() )
meaned_main[ (meaned_main.Minutes >= 58 ) & (meaned_main.Minutes <= 8 )  ].Activity.value_counts()

Standing    126
Walking       5
Name: Activity, dtype: int64

Series([], Name: Activity, dtype: int64)

In [217]:
meaned_main

,Angle,X,Y,Z,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Second,Minutes,Hour,Activity
0,8.192088,-0.988333,0.801667,-9.568333,-0.031250,0.000000,-0.093750,16.114583,-18.776042,-62.453125,2.500000,32.0,14.0,Standing
1,8.192088,-1.008333,0.835833,-9.569167,-0.130208,-0.093750,-0.072917,16.135417,-18.453125,-62.651042,8.666667,32.0,14.0,Standing
2,8.192088,-1.019167,0.846667,-9.560000,0.026042,0.130208,0.072917,16.270833,-18.109375,-62.578125,14.166667,32.0,14.0,Standing
3,7.876297,-1.022500,0.803333,-9.550833,0.343750,-0.078125,0.203125,16.104167,-18.760417,-62.755208,19.500000,32.0,14.0,Standing
4,7.771033,-0.984167,0.787500,-9.556667,0.078125,0.036458,0.130208,16.411458,-18.895833,-62.708333,25.000000,32.0,14.0,Standing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,9.342810,-1.205000,0.921667,-9.662500,0.036458,0.005208,0.062500,22.270833,-14.026042,-69.364583,5.416667,41.0,15.0,Standing
744,9.342810,-1.200833,0.920833,-9.665833,-0.015625,0.052083,-0.031250,22.145833,-13.744792,-69.291667,11.666667,41.0,15.0,Standing
745,9.342810,-1.196667,0.927500,-9.660833,0.010417,-0.026042,-0.005208,22.145833,-13.875000,-69.234375,17.500000,41.0,15.0,Standing
746,9.342810,-1.203333,0.928333,-9.654167,-0.036458,-0.041667,-0.031250,22.203125,-14.093750,-69.187500,23.083333,41.0,15.0,Standing


In [221]:
import  plotly.express as px
fig = px.histogram(meaned_main.loc[285:398], x="Angle", color = 'Activity' , text_auto = True )
fig.show()

import  plotly.express as px
fig = px.histogram(meaned_main[ (meaned_main.Minutes >= 47 ) & (meaned_main.Minutes <= 58 ) & (meaned_main.Hour == 14 ) ], x="Angle", color = 'Activity' , text_auto = True )
fig.show()



In [ ]:
meaned_main[ (meaned_main.Minutes >= 58 ) & (meaned_main.Minutes <= 8 )  ]

In [ ]:
df = encapsule( pd.read_csv( 'dataset_main.csv' , index_col=None ) )